In [1]:
import numpy as np
import pandas as pd
import ast
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv') 

In [3]:
movies = movies.merge(credits,on='title')

In [4]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [5]:
movies.dropna(inplace=True)

In [6]:
def convert(text):
    new_list = []
    for i in ast.literal_eval(text):
        new_list.append(i['name']) 
    return new_list 

In [7]:
movies['genres'] = movies['genres'].apply(convert)

In [8]:
movies['keywords'] = movies['keywords'].apply(convert)

In [9]:
movies['cast'] = movies['cast'].apply(convert)

In [10]:
movies['cast'] = movies['cast'].apply(lambda x:x[0:3])

In [11]:
def get_director(text):
    director_name = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            director_name.append(i['name'])
    return director_name

In [12]:
movies['crew'] = movies['crew'].apply(get_director)

In [13]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [14]:
def collapse(the_list):
    new_list = []
    for i in the_list:
        new_list.append(i.replace(" ",""))
    return new_list

In [15]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

In [16]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [17]:
new = movies.drop(columns=['overview','genres','keywords','cast','crew'])

In [18]:
new['tags'] = new['tags'].apply(lambda x: " ".join(x))

In [19]:
ps = PorterStemmer()

In [20]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [21]:
new['tags'] = new['tags'].apply(stem)

In [22]:
cv = CountVectorizer(max_features=5000,stop_words='english')
vectors = cv.fit_transform(new['tags']).toarray()

In [23]:
similarity = cosine_similarity(vectors)

C:\Users\barda\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [24]:
def recommend(movie):
    index = new[new['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)

In [25]:
import pickle
## pickle.dump(new,open('movie_list.pkl','wb'))

In [26]:
## pickle.dump(new.to_dict(),open('movie_dict.pkl','wb'))


In [27]:
## pickle.dump(similarity,open('similarity.pkl','wb'))